# RNN
- 앞서 배운 신경망 모델들은 '연달아 있는 데이터의 순서와 상호작용을 인식하여 전체 상황을 이해'하는 능력이 없다. (=시간에 대한 개념이 없다.)
- squential data / time series data의 정보를 받아 전체 내용을 학습.
- 정해지지 않은 길이의 배열을 읽고 설명하는 신경망.
- 출력은 순차적 데이터의 흐름을 모두 내포한다.
- 시계열 데이터의 정보를 하나씩 입력받을 때마다 지금까지 입력된 벡터들을 종합해 hidden vector를 만들어낸다.
- 마지막 은닉 벡터는 배열 속 모든 벡터들의 내용을 압축한 벡터.
- 텍스트와 자연어, 주가를 처리하는데 주로 사용된다.
- LSTM (=long short term memory), GRU (=gated recurrent unit)로
- 언어 모델링(=language modeling), 텍스트 감정 분석(=text sentiment analysis), 기계 번역(=machine translation)등에 이용.
- 일대일(일반적인 신경망, CNN).
- 일대다(이미지를 보고 상황을 글로 설명).
- 다대일(감정 분석).
- 다대다1(입력이 끝나고 출력, 챗봇, 기계번역).
- 다대다2(입력되자마자 출력, 매 프레임을 레이블링하는 비디오 분류).

# 프로젝트 1. 영화 리뷰 감정 분석
**RNN 을 이용해 IMDB 데이터를 가지고 텍스트 감정분석을 해 봅시다.**

이번 책에서 처음으로 접하는 텍스트 형태의 데이터셋인 IMDB 데이터셋은 50,000건의 영화 리뷰로 이루어져 있습니다.
각 리뷰는 다수의 영어 문장들로 이루어져 있으며, 평점이 7점 이상의 긍정적인 영화 리뷰는 2로, 평점이 4점 이하인 부정적인 영화 리뷰는 1로 레이블링 되어 있습니다. 영화 리뷰 텍스트를 RNN 에 입력시켜 영화평의 전체 내용을 압축하고, 이렇게 압축된 리뷰가 긍정적인지 부정적인지 판단해주는 간단한 분류 모델을 만드는 것이 이번 프로젝트의 목표입니다.
- 자연어 텍스트는 인공 신경망에 입력시키기 위해 전처리 과정을 거쳐 숫자로 나타내야한다.
- 영화 리뷰를 '언어의 최소 단위'인 토큰으로 나누기.
- 간단한 데이터셋이라면 파이썬의 split() 사용가능.
- 더 좋은 성능을 위해서는 SpaCy 추천.
### 문장 속 모든 토큰을 벡터로 나타내기
1. 데이터셋의 모든 단어(토큰) 수만큼의 벡터들을 담는 dictionary 정의.
2. 토큰을 벡터 형태로 변환(워드 임베딩).

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets
# 토치 텍스트의 전처리 도구들과 파이토치의 nn.Embedding으로 전처리

In [2]:
# 하이퍼파라미터
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("다음 기기로 학습합니다:", DEVICE)

다음 기기로 학습합니다: cpu


In [3]:
# 데이터 로딩하기
print("데이터 로딩중...")

# 텍스트와 라벨을 텐서로 바꿔줄 때 필요한 설정 정하기. 
# sequential : 순차적인 데이터셋인지 명시
# batch_first : 신경망에 입력되는 텐서의 첫 번째 차원값이 batch_size가 되도록 정함.
# lower : 텍스트의 모든 알파벳이 소문자가 되도록 처리.
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

# 모델에 입력되는 데이터셋 만들기.
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

# 워드 임베딩에 필요한 단어 사전(word vocabulary) 만들기.
# min_freq=5 : 데이터에서 최소 5번 이상 등장한 단어만을 사전에 담겠다. 5번 미만으로 출현하는 단어는 unk(=Unknown)토큰으로 대체됨.
TEXT.build_vocab(trainset, min_freq=5)
LABEL.build_vocab(trainset)

# 학습용 데이터를 학습셋 80% 검증셋 20% 로 나누기. 검증셋은 학습의 진행도를 확인하는데 사용.
trainset, valset = trainset.split(split_ratio=0.8)
# 반복 때마다 batch를 생성해주는 반복자iterator를 만들기. 이 반복자를 enumerate() 함수에 입력시켜 루프 구현.
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)

# 사전 속 단어들의 개수와 레이블의 수를 정해주는 변수 만들기.
vocab_size = len(TEXT.vocab)
n_classes = 2

데이터 로딩중...
downloading aclImdb_v1.tar.gz
aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [01:25&lt;00:00, 978kB/s]


In [4]:
print("[학습셋]: %d [검증셋]: %d [테스트셋]: %d [단어수]: %d [클래스] %d"
      % (len(trainset),len(valset), len(testset), vocab_size, n_classes))

[학습셋]: 20000 [검증셋]: 5000 [테스트셋]: 25000 [단어수]: 46159 [클래스] 2


In [5]:
class BasicGRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicGRU, self).__init__()
        print("Building Basic GRU model...")
        # 은닉 벡터들의 층. 아주 복잡한 모델이 아닌 이상 2이하로 정의하는게 일반적. 여러 층이면 multilayer형태의 RNN.
        self.n_layers = n_layers 
        # 첫번째 파라미터 : 전체 데이터셋의 사전에 등재된 단어의 수. 
        # 두번째 파라미터 : 임베딩된 단어 텐서가 지니는 차원값.
        self.embed = nn.Embedding(n_vocab, embed_dim) 
        # 은닉벡터의 차원값.
        self.hidden_dim = hidden_dim
        # 드롭아웃.
        self.dropout = nn.Dropout(dropout_p)
        # 본격적인 RNN 모델 정의.
        # 기본적인 RNN은 앞부분의 정보가 소실되기 때문에, 
        # GRU로 시계열 데이터 속 벡터 사이의 정보 전달량을 조절하여 기울기를 적정하게 유지, 앞부분의 정보가 끝까지 도달할 수 있도록 도움.
        # Update 게이트 : 이전 은닉 벡터가 지닌 정보를 새로운 은닉 벡터가 얼마나 유지할지 정함.
        # Reset 게이트 : 새로운 입력이 이전 은닉 벡터와 어떻게 조합하는지 결정.
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        #시계열 데이터를 압축한 하나의 텐서를 신경망에 통과시켜 클래스에 대한 예측을 출력.
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x) # 한 배치 속에 있는 영화평들을 워드 임베딩.
        h_0 = self._init_state(batch_size=x.size(0)) #첫번째 은닉 벡터를 정의. 직접 정의한 함수를 사용. 보통 모든 특성값이 0인 벡터로 설정.
        # gru에 입력하면 은닉 벡터들이 시계열 배열 형태로 반환됨.
        # 결과값은 (batch_size, 입력 x의 길이, hidden_dim)의 모양을 지닌 3d 텐서.
        x, _ = self.gru(x, h_0)  # [i, b, h]
        h_t = x[:,-1,:] # 인덱싱으로 배치 내 모든 시계열 은닉 벡터들의 마지막 토큰을 내포한 (bach_size, 1, hidden_dim)모양의 텐서를 추출할 수 있음.
        # h_t가 곧 영화 리뷰 배열들을 압축한 은닉 벡터
        self.dropout(h_t) # 드롭아웃 설정
        logit = self.out(h_t)  # [b, h] -> [b, o]
        return logit
    
    def _init_state(self, batch_size=1):
        #parameters() 함수는 nn.Module의 가중치 정보들을 iterator 형태로 반환.
        # 이 반복자가 생성하는 원소들은 각각 실제 신경망의 가중치 텐서(.data)를 지닌 객체들임.
        weight = next(self.parameters()).data 
        # new()함수로 모델의 가중치와 같은 모양으로 변환한 후 zero_()로 모든 값을 0으로 초기화함.
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_() 


In [6]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)  # 레이블 값을 '1과 2' 에서 '0과 1'로 변환.
        optimizer.zero_grad()

        logit = model(x) # 예측값.
        loss = F.cross_entropy(logit, y) #오차
        loss.backward() # 기울기
        optimizer.step() # 최적화

In [7]:
def evaluate(model, val_iter): # 검증셋과 테스트셋 성능 측정
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0 
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환

        logit = model(x)
        #업데이트 없음.

        loss = F.cross_entropy(logit, y, reduction='sum') # 오찻값과 정확도를 구하기 위해 배치의 평균이 아닌 합을 구함.
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum() # 정답 맞춘 것들 합.
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy # 오찻값과 정확도의 평균.

In [8]:
model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE) # 모델 객체 정의. 은닉 벡터는 256차원. 임베딩된 토큰은 128차원.
optimizer = torch.optim.Adam(model.parameters(), lr=lr) # Adam 최적화 알고리즘 사용.
"""
Building Basic GRU model...
BasicGRU(
  (embed): Embedding(46159, 128)
  (dropout): Dropout(p=0.5)
  (gru): GRU(128, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=2, bias=True)
)
"""

Building Basic GRU model...


In [9]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[이폭: %d] 검증 오차:%5.2f | 검증 정확도:%5.2f" % (e, val_loss, val_accuracy))
    
    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot") # 없으면 만들기
        torch.save(model.state_dict(), './snapshot/txtclassification.pt') # 저장
        best_val_loss = val_loss

KeyboardInterrupt: 

In [10]:
#테스트 셋으로 시험.
model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))

테스트 오차:  0.31 | 테스트 정확도: 86.00
